# Adrien Hans
# Projet d'intégration : Exploitation des données du grand débat national 

imports : 

In [1]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from collections import defaultdict

#Pour créer les cartes :
import folium
from folium.plugins import MarkerCluster
from folium.plugins import HeatMap
import geopandas

4 grands thèmes dans la base de données : 
    - Transition Ecologique
    - Organisation de l'état et des services publics
    - La fisclité et les dépenses publiques
    - Démocratie et citoyenneté

## Transition Ecologique : 

In [2]:
data_Ecolo = pd.read_csv('LA_TRANSITION_ECOLOGIQUE.csv')

FileNotFoundError: [Errno 2] File b'LA_TRANSITION_ECOLOGIQUE.csv' does not exist: b'LA_TRANSITION_ECOLOGIQUE.csv'

### Description succinte des données : 

In [ ]:
data_Ecolo.shape

In [ ]:
data_Ecolo.head()

In [ ]:
data_Ecolo.info()

In [ ]:
data_Ecolo.describe(include='all')

In [ ]:
data_Ecolo.isna().sum()

## Organisation de l'état et des services publics : 

In [ ]:
data_serv_pub = pd.read_csv('ORGANISATION_DE_LETAT_ET_DES_SERVICES_PUBLICS.csv',error_bad_lines=False)

### Description succinte des données : 

In [ ]:
data_serv_pub.shape

In [ ]:
data_serv_pub.head()

In [ ]:
data_serv_pub.info()

In [ ]:
data_serv_pub.describe(include='all')

In [ ]:
data_serv_pub.isna().sum()

## La Fiscalité et les dépenses publiques :

In [ ]:
data_fisc = pd.read_csv('LA_FISCALITE_ET_LES_DEPENSES_PUBLIQUES.csv')

### Description succinte des données : 

In [ ]:
data_fisc.shape

In [ ]:
data_fisc.head()

In [ ]:
data_fisc.info()

In [ ]:
data_fisc.describe(include='all')

In [ ]:
data_fisc.isna().sum()

## Démocratie et citoyenneté : 

In [ ]:
data_dem = pd.read_csv('DEMOCRATIE_ET_CITOYENNETE.csv')

### Description succinte des données :

In [ ]:
data_dem.shape

In [ ]:
data_dem.head()

In [ ]:
data_dem.info()

In [ ]:
data_dem.describe(include='all')

In [ ]:
data_dem.isna().sum()

## Bases des codes postaux et visualisation sur une carte

Comme on l'a vu, les codes postaux sont renseignés avec chaque contribution. 

On peut donc essayer, dans un premier temps, de visualiser quels codes postaux figurent dans la base de donnée. 

Puis, dans un second temps, on peut essayer de faire une "heatmap" des endroits qui ont le plus contribué au grand débat. 

Pour ça, on peut utiliser la librairie `folium` https://python-visualization.github.io/folium/quickstart.html

On peut aussi utiliser, ce sera utile de toute façon, récupérer la base de données officielle des codes posatux fournie par l'état. 
Cette base donne à la fois le nom des communes, le code postal associé, le code INSEE et surtout, pour la visualisation, les coordonnées GPS. 
Je l'ai trouvée ici : https://www.data.gouv.fr/fr/datasets/base-officielle-des-codes-postaux/#_

#### Ouverture de la base de donnée officielle des codes postaux : 

In [ ]:
Data_Zip_Code = pd.read_csv('laposte_hexasmal.csv',';')
Data_Zip_Code.head()

Je me suis rendu compte qu'il y avait des nans dans cette base de données à la place des coordonnées gps. 

Pour pouvoir exploiter des première cartes, j'ai décidé d'attribuer les coordonnées : `45.716059, -2.885733`qui sont dans l'océan atlantique (cela est une solution basique permettant la visualisation des premières cartes malgré tout. 

In [ ]:
Data_Zip_Code['coordonnees_gps'] = Data_Zip_Code['coordonnees_gps'].fillna("45.716059, -2.885733",inplace=False)

In [ ]:
Data_Zip_Code.isna().sum()

In [ ]:
Data_Zip_Code

Etant données les natures des bases de données, il faudrait prendre chaque code postal dans les bases de données du Grand débat, rechercher ses coordonnees gps via ce dernier fichier, et les afficher avec `folium`

On peut aussi utiliser `basemap` ou bien `geopandas`.


#### Création de la carte de France avec `folium` :

In [ ]:
"""
locations = df_villes[['Latitude', 'Longitude']].copy()
print(locations.shape, locations.dropna().shape)
locations.dropna(inplace = True)

locationlist = locations.values.tolist()
len(locationlist)
locationlist[7]

communes_random = random.sample(locationlist, 50)
"""

map = folium.Map(location=[47.088615, 2.637424], zoom_start=6)

#for point in range(0,len(communes_random)) :
#    folium.Marker(communes_random[point]).add_to(map)
map

On rassemble toutes les contributions dans un seul objet pour récupérer tous les codes postaux des contributions. 

In [ ]:
data_Ecolo_copy=data_Ecolo['authorZipCode'].copy()
data_serv_pub_copy=data_serv_pub['authorZipCode'].copy()
data_fisc_copy=data_fisc['authorZipCode'].copy()
data_dem_copy=data_dem['authorZipCode'].copy()

CP_contrib=pd.concat([data_Ecolo_copy,data_fisc_copy,data_dem_copy],ignore_index=True)

**Je n'ai pas pris en compte la base service public car il y a des valeurs textuelles dans la colonne code postal. Il faudra la réintégrer en corrigeant ceci.**

In [ ]:
len(CP_contrib)

Tous les codes postaux (sans vérifier leur unicité) sont donc dans `CP_contrib`

On peut essayer ne prendre que ceux qui sont en France métropolitaine et les trier par nombre d'occurences.

Les premier département français métropolitain étant l'Ain avec pour code postal 01xxx et le dernier (en métropole selon la numérotation des départements) étant le val d'Oise, avec pour code postal 95xxx

On va essayer de ne sélectionner que les codes postaux avec des valeurs numériques situées entre 1000 et 96 000.

In [ ]:
#CP_contrib.values.tolist()
#CP_contrib.values
CP_contrib=CP_contrib.to_frame()

In [ ]:
#On convertit en entiers:
CP_contrib.astype(np.int64).info()

In [ ]:
#On ne selectionne que les codes postaux situés entre 1000 et 96000 :
CP_contrib=CP_contrib.loc[CP_contrib['authorZipCode']>=1000]
CP_contrib=CP_contrib.loc[CP_contrib['authorZipCode']<96000]
CP_contrib.info()

In [ ]:
#On compte chaque code postal : 
#CP_contrib_count = CP_contrib['authorZipCode'].value_counts().to_frame(name='count')

#CP_contrib_count.index.name='authorZipCode'
#CP_contrib_count

On voit par exemple que le code postal avec le plus de contributions est le 15ème arrondissement parisien. 
Cela semble relativement logique. 

In [ ]:
CP_contrib

In [ ]:
#On créée un DataFrame pour associer code postal et coordonnées gps : 
CP_GPS = pd.DataFrame({'CP':Data_Zip_Code['Code_postal'], 'GPS':Data_Zip_Code['coordonnees_gps']})
CP_GPS_d = CP_GPS.set_index('CP').to_dict()['GPS']

In [ ]:
#On remplace les codes postaux par les coordonnées GPS : 
#CP_contrib = CP_contrib.replace(CP_GPS_dict)

#On utilise finalement la deuxième méthode, plus stable que ce qui avait été testé précedemment, en 'mappant' la dataFrame
#On utilise toujours la valeur par défaut dans l'océan Atlantique pour l'instant

CP_GPS_dict = defaultdict(lambda: '45.716059,-2.885733')
CP_GPS_dict.update(CP_GPS_d)
CP_contrib['authorZipCode']= CP_contrib['authorZipCode'].map(CP_GPS_dict)

#### Erreurs et nettoyage : 
Certaines valeurs de code postal ne trouvent pas de coordonnées GPS. 
Ainsi, il faut soit :
- Remplacer ces valeurs par des coordonnées arbitraires (comme précédemment)
- Remplacer les codes postaux par le code postal le plus proche

In [ ]:
#CP_contrib['authorZipCode'].fillna("45.716059,-2.885733",inplace=True)
CP_contrib.iloc()[453]

In [ ]:
CP_contrib

In [ ]:
CP_contrib['authorZipCode']=CP_contrib['authorZipCode'].str.split(',')

#On convertit en list pour pouvoir afficher avec folium :
CP_contrib_list_folium=CP_contrib['authorZipCode'].tolist()

#### Cartes : 

In [ ]:
map = folium.Map(location=[47.088615, 2.637424], zoom_start=6)
for point in range(0,2000):
    folium.Marker(CP_contrib_list_folium[point]).add_to(map)
map

In [ ]:
map2 = folium.Map(location=[47.088615, 2.637424], zoom_start=6)
marker_cluster = folium.plugins.MarkerCluster().add_to(map2)
for point in range(0, 2000):
    folium.Marker(CP_contrib_list_folium[point]).add_to(marker_cluster)
map2

In [ ]:
m = folium.Map(location=[47.088615, 2.637424],zoom_start=6)
HeatMap(CP_contrib_list_folium[0:2000]).add_to(map2)
map2